<a href="https://colab.research.google.com/github/aerosense-ai/notebooks/blob/main/Battery_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!pip install --upgrade git+https://github.com/aerosense-ai/aerosense-tools.git@0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/aerosense-ai/aerosense-tools.git (to revision 0.4.0) to /tmp/pip-req-build-qtt6coms
  Running command git clone --filter=blob:none --quiet https://github.com/aerosense-ai/aerosense-tools.git /tmp/pip-req-build-qtt6coms
  Resolved https://github.com/aerosense-ai/aerosense-tools.git to commit 9b806cb2a450f9ef811e33e48f5b560bf57342dd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 KB 16.2 MB/s eta 0:00:00
  Created wheel for aerosense-tools: filename=aerosense_tools-0.4.0-py3-none-any.whl size=9236 sha256=260c9026a1bd47c8d34f211905fb53bc1b3ce84d4ab7691bf1d76e946a5350e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-xirc1d9y/wheels/3e/ee/b0/

In [3]:
import datetime as dt

from aerosense_tools.queries import BigQuery
from aerosense_tools import plots
from aerosense_tools.preprocess import RawSignal
from aerosense_tools.preprocess import SensorMeasurementSession

client = BigQuery()

In [7]:
sensor_types_metadata = {
    "accelerometer": {
        "description": "Accelerometer Measurement Series",
        "variable": "Acceleration [m/s²]",
        "sensors": ["Acc. Direction 1", "Acc. Direction 2", "Acc. Direction 3"],
        },
    "barometer": {
        "description": "Barometer Measurement Series",
        "variable": "Pressure [Pa]",
        "sensors": ["Abs. Baro. Sensor {}".format(i) for i in range(40)],
    },
    "barometer_thermometer": {
        "description": "Temperature Measurement Series from Barometer Sensors",
        "variable": "Temperature [C]",
        "sensors": ["Temp. Sensor {}".format(i) for i in range(40)]
    },
    "connection_statistics": {
        "description": "Connection Statistics",
        "variable": [
            "Transmitting Power [W]",
            "Allocated Heap Memory [Bytes]",
            "Received signal strength indication: Raw [dBm]",
            "Received signal strength indication: Filtered [dBm]"
        ],
        "sensors": ["Conn. Stat. {}".format(i) for i in range(4)]
    },
    "battery_info": {
        "description": "Battery State information",
        "variable": [
            "Voltage [V]",
            "Number of cycles",
            "Battery Capacity [%]"
        ],
        "sensors": ["Batt. info. {}".format(i) for i in range(3)]
    },
    "differential_barometer": {
        "description": "Differential Pressure Measurement Series",
        "variable": "Differential Pressure [Pa]",
        "sensors": ["Diff. Baro. Sensor {}".format(i) for i in range(5)],
        },
    "gyroscope": {
        "description": "Gyroscope Measurement Series",
        "variable": "Angular Velocity [s⁻¹]",
        "sensors": ["Gyro. Direction 1", "Gyro. Direction 2", "Gyro. Direction 3"],
        },
    "magnetometer": {
        "description": "Magnetometer Measurement Series",
        "variable": "Magnetic Momemnt [A⋅m²]",
        "sensors": ["Mag. Direction 1", "Mag. Direction 2", "Mag. Direction 3"],
        },
}

# Battery Information

In [8]:
node="1"
installation = "aventa-turbine-test"
measurement_day = dt.datetime(2023, 1, 1)
query_period = dt.timedelta(days=365)

df, data_limit_applied = client.get_sensor_data(
    installation_reference=installation,
    node_id=node,
    sensor_type_reference="battery_info",
    start=measurement_day,
    finish=measurement_day+query_period,
    row_limit=10000000,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
info_df = df[["datetime"]+data_columns].set_index('datetime')
info_df.columns = sensor_types_metadata["battery_info"]["variable"]

In [12]:
import plotly.express as px

info_df.drop(["Number of cycles"], axis=1, inplace=True)

layout={
    "xaxis_title":"Date/Time"
}
for variable in info_df.columns:
  figure = px.line(info_df, y=variable)
  figure.update_layout(layout)
  figure.show()